# Generate model response

In [ ]:
import os
import dotenv
import sys

dotenv.load_dotenv(".env")
sys.path.append(os.path.join(os.getcwd(), "..", "src"))

### Custom RAG Queries

In [ ]:
from orchestration.utilities import OrchestrationClient

# Create orchestration client
orchestration_client = OrchestrationClient(
    open_ai_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    open_ai_chat_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
    open_ai_embedding_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
    search_endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    search_index_name=os.getenv("AZURE_AI_SEARCH_INDEX_NAME"),
)

In [ ]:
# Generate chat response from initial user query
chat_history = {
    "messages": [
        {"role": "user", "content": "Which tent is the most waterproof?"},
    ]
}

chat_history = orchestration_client.generate_chat_response(chat_history)
print(chat_history["messages"][-1]["content"])

In [ ]:
# Generate chat response from follow-up user query
chat_history["messages"].append(
    {"role": "user", "content": "Tell me more about the Alpine Explorer Tent?"}
)

chat_history = orchestration_client.generate_chat_response(chat_history)
print(chat_history["messages"][-1]["content"])

### Azure OpenAI Service REST API

Note: this will require public access on Azure AI Search or a Microsft managed private endpoint for private access.

In [ ]:
from azure.identity import DefaultAzureCredential
import requests

credential = DefaultAzureCredential()
access_token = credential.get_token("https://cognitiveservices.azure.com/.default")

open_ai_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
open_ai_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
open_ai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

chat_endpoint = f"{open_ai_endpoint}/openai/deployments/{open_ai_chat_deployment}/extensions/chat/completions?api-version={open_ai_api_version}"

request_headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token.token}",
}

request_payload = {
    "dataSources": [
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
                "indexName": os.getenv("AZURE_AI_SEARCH_INDEX_NAME"),
                "queryType": "vectorSemanticHybrid",
                "embeddingDeploymentName": os.getenv(
                    "AZURE_OPENAI_EMBEDDING_DEPLOYMENT"
                ),
                "fieldsMapping": {"titleField": "title", "urlField": "path"},
            },
        }
    ],
    "messages": [{"role": "user", "content": "Which tent is the most waterproof?"}],
}

response = requests.post(
    chat_endpoint,
    headers=request_headers,
    json=request_payload,
)

print(response.json())

In [ ]:
text = "Based on the information provided, both the Alpine Explorer Tent and the TrailMaster X4 Tent are waterproof. The Alpine Explorer Tent has a rainfly with a waterproof rating of 3000mm [product_info_8.md], while the TrailMaster X4 Tent has a rainfly with a waterproof rating of 2000mm [product_info_1.md]. Therefore, both tents offer reliable protection against rain and moisture."

text

In [ ]:
import re

text = (
    "This document refers to [product_info_1.md] and [another_file.md]. More text here."
)

def replace_references(text: str) -> str:
    # Regex to match references in the format [*.md]
    regex = r"\[([^\]]*.md)\]"

    # Replace matched references with modified references (appending ":blue")
    modified_text = re.sub(regex, r"*:blue[\1]*", text)

    return modified_text

# Regex to match references in the format [*.md]
regex = r"\[([^\]]*.md)\]"

# Replace matched references with modified references (appending ":blue")
modified_text = re.sub(regex, r"*:blue[\1]*", text)

print(modified_text)